In [ ]:
"""
VFGLVM: Multi-Frequency Comparison (Daily, Weekly, Monthly)
===========================================================
Gold ETF Market Cap vs Bitcoin Market Cap ONLY

This script implements the Variable-Order Fractional Grey
Lotka–Volterra Model (VFGLVM) in the style of Gatabazi et al. (2019),
but adapted to gold-vs-bitcoin capital allocation instead of
crypto transaction counts.

High-level pipeline (ties directly to thesis Sections 3.2.*):

Step 0. Data prep (OUR extension)
    - Build Gold ETF AUM in USD and Bitcoin market cap in USD.
    - Align to daily, optionally resample to weekly/monthly.
    - Winsorize Bitcoin extremes, interpolate gold holdings.
    - Median-scale each series so regression is numerically stable.
      (Not in Gatabazi et al. 2019; needed here because the two assets live
       on very different dollar scales.)

Step 1. Discretize Lotka–Volterra
    We use a one-step-ahead forecast form instead of continuous-time ODEs.
    x_i(k+1) ~ f(Z_i(k), Z_j(k))
    This matches the discrete VFGLVM forecasting equation.

Step 2. Fractional accumulation (long memory)
    We transform each raw series x_i into a memory-adjusted series
    X_i^{[q(t)]}(k) using variable-order fractional accumulation with
    Gamma-based weights.

Step 3. Variable-order q(t) (adaptive memory)
    We estimate q(t) from local slopes/volatility: fast, jumpy periods
    => shorter memory; calm periods => longer memory. We then clip
    q(t) into [0.3,0.7] and smooth it.
    (We use a continuous q(t); Gatabazi et al. later summarize q(t)
     piecewise by regime.)

Step 4. Background value Z_i(k) (short-term stabilizer)
    Z_i(k) = 0.5 * ( X_i^{[q(t)]}(k) + X_i^{[q(t)]}(k-1) )
    This filters out one-step jitters and gives the "effective state"
    of each asset at time k.

Step 5. Fit VFGLVM parameters and forecast
    For each asset i:
       x_i(k+1) ≈ a_i Z_i(k)
                  - b_i [Z_i(k)]^2
                  - Σ_{j≠i} C_ij Z_i(k) Z_j(k)
    We estimate (a_i, b_i, C_ij) via (ridge-stabilized) least squares.
    Gatabazi et al. (2019) use plain least squares; we add a tiny ridge
    alpha=1e-5 just to avoid numerical blow-ups.

After fitting we:
    - Re-scale fitted values back to USD.
    - Compute MAPE (as in Gatabazi et al. 2019), and also RMSE in $B.
    - Plot Gold vs Bitcoin actual vs fitted at Daily / Weekly / Monthly.
    - Dump summary tables.

NOTE: We do not currently reproduce chaos diagnostics
(Lyapunov exponents, phase portraits).
"""

from __future__ import annotations
import math, json
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.special import gammaln

In [ ]:
# ============================================================================
# CONFIG / OUTPUT PATHS
# ============================================================================

START_DATE = "2016-01-01"
END_DATE   = "2025-09-30"

# Raw inputs:
# - Gold ETF holdings in ounces (monthly or daily-ish file)
# - LBMA gold price in USD/oz (daily JSON)
# - Bitcoin market cap in USD (TSV with BTC / Market Cap (USD))
PATH_GOLD_ETF   = r"C:\Users\Kevin\Downloads\Thesis\Data\Gold Volume.xlsx"
PATH_LBMA_JSON  = r"C:\Users\Kevin\Downloads\Thesis\Data\LBMA gold price 12.10.2025.json"
PATH_BTC_TSV    = r"C:\Users\Kevin\Downloads\Thesis\Data\coin-metrics-new-chart.tsv.tsv"

# Winsorization on BTC cap to reduce flash-crash style outliers.
WINSOR_PCT = 0.005

OUTDIR = Path.cwd() / "output" / "vfglvm_multi_frequency_btc_only"
OUTDIR.mkdir(parents=True, exist_ok=True)

# Plot style
mpl.rcParams.update({
    "figure.dpi": 100,
    "axes.titlesize": 11,
    "axes.labelsize": 10,
    "xtick.labelsize": 9,
    "ytick.labelsize": 9,
    "grid.alpha": 0.3,
    "grid.linestyle": ":",
})
plt.style.use("seaborn-v0_8-darkgrid")

In [ ]:
# ============================================================================
# STEP 1: DISCRETE LOTKA–VOLTERRA SETUP
# ----------------------------------------------------------------------------
# THESIS REFERENCE: Section 3.2, Step 1
# 
# Instead of continuous dX/dt = ..., we forecast one step ahead directly:
#
#   x_i(k+1) ≈ a_i Z_i(k) - b_i [Z_i(k)]^2 - Σ_{j≠i} C_ij Z_i(k) Z_j(k)
#
# where Z_i(k) is a smoothed/filtered state (Step 4), not the raw level.
# 
# WHY THIS MATTERS:
# Classic Lotka-Volterra uses continuous-time differential equations suitable
# for stable biological populations. Financial markets have:
#   - Discrete observations (daily, weekly, monthly)
#   - Regime shifts and discontinuities
#   - Noise that makes derivatives unstable
# 
# The discrete forecasting form allows VFGLVM to:
#   1. Work directly with observed time series (no derivative approximation)
#   2. Make testable one-step-ahead predictions
#   3. Incorporate memory-adjusted states Z_i(k) instead of raw prices
#
# This is how VFGLVM makes LV usable on noisy financial data.
# Matches Gatabazi et al. (2019) Eq. (15).
# ============================================================================

def _as_array(x) -> np.ndarray:
    return np.asarray(x, dtype=float)

def _ridge(X: np.ndarray, y: np.ndarray, alpha: float = 1e-6) -> np.ndarray:
    """
    Small ridge-regularized least squares.
    
    Gatabazi et al. (2019) solve with vanilla least squares: θ̂ = (B'B)^(-1)B'M
    
    We add alpha on the diagonal for numerical stability because Z_i, Z_i^2,
    and Z_i*Z_j can be highly collinear in real market data.
    
    WHY RIDGE IS NEEDED (our extension):
    1. COLLINEARITY: When gold and crypto trend together, Z_gold and Z_btc
       become correlated, making (B'B) near-singular
    2. SCALE DIFFERENCES: Even after median-scaling, variance properties differ
    3. NUMERICAL SAFETY: α=1e-5 is negligible for parameter bias but prevents
       matrix inversion failures
    
    This is a standard numerical practice when applying theoretical models
    to real financial data.
    """
    XtX = X.T @ X
    XtX[np.diag_indices_from(XtX)] += alpha
    Xty = X.T @ y
    return np.linalg.solve(XtX, Xty)

In [ ]:
# ============================================================================
# STEP 2: VARIABLE-ORDER FRACTIONAL ACCUMULATION
# ----------------------------------------------------------------------------
# THESIS REFERENCE: Section 3.2, Step 2
# 
# Given raw series x_i(k) we build
#
#   X_i^{[q(t)]}(k) = sum_{m=0..k} w_{k,m}( q(k) ) * x_i(m)
#
# where the weights w_{k,m} are Gamma-based. This is the discrete
# variable-order fractional accumulation described in Gatabazi et al. (2019)
# Eq. (10).
# 
# INTUITION: This is a "soft memory curve"
# - Recent points (m close to k) receive HIGH weight
# - Older points (m far from k) receive LOWER weight (exponential fade)
# - Nothing is thrown away completely (unlike moving averages)
#
# WHY THIS MATTERS FOR FINANCE:
# Financial markets have LONG MEMORY:
#   - Past bull markets influence current investor psychology
#   - Historical volatility affects risk premia today
#   - Previous regime characteristics carry forward
#
# Standard models (AR, MA) use FIXED exponential decay or truncation.
# Fractional accumulation uses FLEXIBLE Gamma-weighted memory that:
#   1. Adapts its decay rate via q(t)
#   2. Never fully "forgets" the past (tail never hits zero)
#   3. Produces smoother state variables for LV dynamics
#
# This implements grey system theory's core principle: extract stable
# signals from noisy, limited data by controlled memory accumulation.
# ============================================================================

def fractional_accum_variable_order(x: np.ndarray,
                                    q_t: np.ndarray,
                                    max_lag: int) -> np.ndarray:
    """
    Build X_i^{[q(t)]}(k) using variable-order fractional accumulation.
    We only sum back `max_lag` steps for efficiency/stability.

    Gamma ratio:
       coeff = Γ(q + dist) / [ Γ(q) Γ(dist+1) ]
    matches fractional accumulation weights in Gatabazi et al. (2019) Eq. (9).
    
    PARAMETERS:
    - x: Raw time series (e.g., gold market cap in scaled units)
    - q_t: Time-varying memory order (from Step 3)
    - max_lag: How far back to accumulate (computational efficiency)
    
    RETURNS:
    - X^{[q(t)]}: Memory-adjusted series incorporating weighted history
    """
    n = len(x)
    if len(q_t) != n:
        raise ValueError("q_t must have same length as x")

    out = np.zeros(n, dtype=float)
    for k in range(n):
        # Local memory order at time k (varies over time)
        qk = float(np.clip(q_t[k], 0.05, 0.95))

        # Don't sum from 0..k if k is huge: cap by max_lag
        i_start = max(0, k - max_lag + 1)
        acc = 0.0
        for i in range(i_start, k+1):
            dist = k - i  # How far back in time
            # Log Gamma ratio to avoid under/overflow
            lg = gammaln(qk + dist) - gammaln(qk) - gammaln(dist + 1.0)
            coeff = math.exp(lg)
            acc += coeff * x[i]
        out[k] = acc
    return out


In [ ]:
# ============================================================================
# Helper for rolling slope (used in q(t) estimation, Step 3)
# ============================================================================
def _rolling_slope(y: np.ndarray, window: int) -> np.ndarray:
    """
    Fit simple slopes over a rolling window.
    
    INTUITION:
    - Big slope/vol spikes => the process is moving fast/jumpy "right now"
    - We use that to shrink memory length q(t) in those regions
    
    This follows the idea in Gatabazi et al. (2019): q(t) reflects how fast
    the underlying process is changing; faster change => different memory
    regime.
    
    ECONOMIC INTERPRETATION:
    - Steep slopes = rapid capital reallocation = shortened memory
    - Flat slopes = stable regime = extended memory
    """
    n = len(y)
    out = np.full(n, np.nan)
    t = np.arange(n, dtype=float)
    for end in range(window - 1, n):
        start = end - window + 1
        yy = y[start:end+1]
        tt = t[start:end+1]
        mask = np.isfinite(yy)
        if mask.sum() < max(2, window // 2):
            continue
        yyv = yy[mask]
        ttv = tt[mask]
        ym = np.mean(yyv)
        tm = np.mean(ttv)
        num = np.sum((ttv - tm) * (yyv - ym))
        den = np.sum((ttv - tm)**2)
        out[end] = np.nan if den == 0 else num / den
    return out

# ============================================================================
# STEP 3: ESTIMATE q(t) (ADAPTIVE MEMORY ORDER)
# ----------------------------------------------------------------------------
# THESIS REFERENCE: Section 3.2, Step 3
#
# q(t) controls how much past information we keep in Step 2.
# 
# ECONOMIC INTUITION:
# - When recent data are JUMPY/VOLATILE → SHORT memory (lower q)
#   Example: 2020 COVID crash, 2021 crypto mania
#   The system shouldn't trust old "normal" data in chaotic regimes
#
# - When recent data are CALM/TRENDING → LONG memory (higher q)
#   Example: 2017-2019 stable gold range
#   The system can safely incorporate historical context
#
# IMPLEMENTATION:
# We:
#   (a) Compute rolling slopes for each series (rate of change)
#   (b) Take absolute values and average across assets → aggregate instability
#   (c) INVERT the instability signal: high instability → low q
#   (d) Clip to [0.3, 0.7] to prevent extreme memory behavior
#   (e) Smooth q(t) across time to avoid discontinuous jumps
#
# DESIGN CHOICE: Continuous vs Piecewise q(t)
# -------------------------------------------
# Gatabazi et al. (2019) report PIECEWISE CONSTANT q(t) by regime:
#   - Example from their Table: q=0.0196 for t≤1665, q=0.2717 for t>1665
#   - This requires identifying regime break-points (structural break tests)
#
# We use CONTINUOUS q(t) instead, which:
#   ✓ Adapts smoothly without arbitrary break-point selection
#   ✓ Responds granularly to market phases
#   ✓ Avoids overfitting to specific historical regimes
#   ✗ May be less interpretable (no discrete "regime labels")
#
# Both approaches are valid. Continuous q(t) is more flexible for
# out-of-sample forecasting where future regime breaks are unknown.
#
# This matches the spirit of Gatabazi et al. (2019) Section 2.1,
# where they emphasize q(t) must vary with observed dynamics.
# ============================================================================

def estimate_q_t(series_matrix: np.ndarray,
                 window: int,
                 clip: Tuple[float,float]=(0.3,0.7),
                 smooth_ma: int=4) -> np.ndarray:
    """
    Estimate time-varying memory order q(t) from data dynamics.
    
    PARAMETERS:
    - series_matrix: (n_assets, n_periods) array of SCALED asset levels
    - window: Rolling window for slope estimation (market dynamics detector)
    - clip: (min_q, max_q) bounds for memory order
    - smooth_ma: Smoothing window to prevent q(t) discontinuities
    
    RETURNS:
    - q_t: (n_periods,) array of adaptive memory orders
    
    INTERPRETATION:
    - q_t[k] ≈ 0.3 → Market is volatile, use SHORT memory
    - q_t[k] ≈ 0.7 → Market is stable, use LONG memory
    - q_t[k] in between → Transitional regime
    """
    n, k = series_matrix.shape

    # Safety tweak: avoid absurdly large windows
    if window > k // 2:
        window = max(4, k // 4)

    # (a) Compute rolling slopes for each asset (rate of change measure)
    slopes_abs = []
    for i in range(n):
        s = _rolling_slope(series_matrix[i], window)
        slopes_abs.append(np.abs(s))

    # (b) Aggregate instability across assets (market-wide volatility)
    S = np.nanmean(np.vstack(slopes_abs), axis=0)

    valid = S[np.isfinite(S)]
    if len(valid) < 5:
        # Fallback: constant mid-range memory if insufficient data
        q = np.full(k, np.mean(clip))
    else:
        # (c) INVERT instability: high instability → low q
        # Normalize instability to [0, 1] range
        S_min = np.nanmin(valid)
        S_max = np.nanmax(valid)
        
        if S_max > S_min:
            # Normalized instability in [0, 1]
            S_norm = (S - S_min) / (S_max - S_min)
            S_norm = pd.Series(S_norm).ffill().bfill().values
            
            # INVERT: high instability (1) → low q (clip[0]=0.3)
            #         low instability (0) → high q (clip[1]=0.7)
            q = clip[1] - S_norm * (clip[1] - clip[0])
        else:
            # No variation in instability → constant memory
            q = np.full(k, np.mean(clip))

        # (d) Ensure bounds (already satisfied by construction, but enforce)
        q = np.clip(q, clip[0], clip[1])

    # (e) Smooth q(t) across time to prevent discontinuous jumps
    # Financial insight: Memory characteristics shouldn't flip overnight
    if smooth_ma and smooth_ma > 1:
        q = pd.Series(q).rolling(smooth_ma, min_periods=1,
                                 center=True).mean().values
        q = np.clip(q, clip[0], clip[1])

    return q


In [ ]:
# ============================================================================
# STEP 4: BACKGROUND VALUE CONSTRUCTION
# ----------------------------------------------------------------------------
# THESIS REFERENCE: Section 3.2, Step 4
#
# After fractional accumulation we still smooth once more:
#
#   Z_i(k) = 0.5 * ( X_i^{[q(t)]}(k) + X_i^{[q(t)]}(k-1) )
#
# Gatabazi et al. call this the "mean sequence" or "background value".
# 
# PURPOSE: Kill single-interval jumps while keeping medium-run structure
#
# WHY THIS IS NEEDED (Financial Market Reality):
# ------------------------------------------------
# Financial markets exhibit TWO TYPES of instability:
#
# 1. REGIME-LEVEL VOLATILITY (captured by q(t) in Step 3)
#    Example: 2020 COVID crash changes market dynamics for months
#    → q(t) drops system-wide, model uses shorter memory
#
# 2. TICK-LEVEL NOISE (filtered by Z_i(k) in Step 4)
#    Example: Elon Musk tweets about Bitcoin at 3am
#    → BTC jumps 8% in one hour, but doesn't reflect capital reallocation
#
# Even after fractional accumulation smooths long-term memory,
# X_i^{[q(t)]}(k) can still contain short-term jitter from:
#   - Flash crashes or single-tweet events in crypto
#   - Gap openings between trading sessions in gold
#   - High-frequency trading noise and algorithmic herding
#   - Month-end rebalancing spikes
#
# The background value Z_i(k) acts as a SHORT-TERM stabilizer:
#
#   Z_i(k) = 0.5 * (X_i^{[q(t)]}(k) + X_i^{[q(t)]}(k-1))
#
# This averaging:
#   ✓ Preserves medium-run trends (bull/bear markets)
#   ✓ Preserves memory structure from Step 2
#   ✓ Filters single-interval shocks (one-day events)
#   ✓ Produces smoother "effective state" for LV dynamics
#
# RESULT: Z_i(k) represents "where capital is actually positioned"
# after removing both long-term noise (via memory) and short-term
# noise (via consecutive averaging).
#
# Gatabazi et al. (2019) Eq. (14) emphasizes this prevents one-step
# jumps from distorting the LV interaction structure.
# ============================================================================

def build_Z_q(levels: List[np.ndarray],
              q_t: np.ndarray,
              max_lag: int) -> Tuple[List[np.ndarray], List[np.ndarray]]:
    """
    Build background values Z_i(k) for each asset.
    
    PROCESS:
    1. Apply fractional accumulation → X_i^{[q(t)]}(k) (long memory)
    2. Average consecutive terms → Z_i(k) (short-term stabilization)
    
    PARAMETERS:
    - levels: list of raw scaled series [x_gold_scaled, x_btc_scaled]
    - q_t: adaptive memory order over time (from Step 3)
    - max_lag: memory horizon used in accumulation (computational bound)
    
    RETURNS:
    - Xq: list of fractionally accumulated series
    - Zq: list of background values (smoothed states for LV dynamics)
    """
    # Step 2: Fractional accumulation for each asset i → X_i^{[q(t)]}(k)
    Xq = [fractional_accum_variable_order(x, q_t, max_lag=max_lag)
          for x in levels]

    # Step 4: Background value Z_i(k) = average of consecutive accumulated terms
    # This removes single-interval noise while preserving trends
    Zq = []
    for xq in Xq:
        z = np.full_like(xq, np.nan)
        z[1:] = 0.5 * (xq[1:] + xq[:-1])
        Zq.append(z)

    return Xq, Zq

@dataclass
class VFGLVMParams:
    """
    Stores fitted parameters of the VFGLVM:
    
    a[i]:     Intrinsic growth rate of asset i
              Economic interpretation: How strongly capital flows into asset i
              when other assets are absent (standalone attractiveness)
              
    b[i]:     Self-saturation coefficient of asset i  
              Economic interpretation: Diminishing returns as asset i grows
              (crowding, liquidity constraints, profit-taking pressure)
              
    C[i,j]:   Cross-effect of asset j on asset i
              Economic interpretation:
              - C[i,j] > 0: Asset j crowds out asset i (competition)
              - C[i,j] < 0: Asset j helps asset i (co-movement, complementarity)
              - C[i,j] ≈ 0: Assets are independent
    
    These match the LV framework: growth, self-limitation, interaction.
    See Gatabazi et al. (2019) Section 2.2 for formal definitions.
    """
    a: np.ndarray
    b: np.ndarray
    C: np.ndarray


In [ ]:
# ============================================================================
# STEP 0: DATA PREP / PANEL CONSTRUCTION (OUR EXTENSION)
# ----------------------------------------------------------------------------
# We are not modeling transaction counts. We are modeling *capital allocation*:
#   - Gold series: Gold ETF AUM in USD
#   - Bitcoin series: BTC market cap in USD
#
# We:
#   - interpolate gold ETF ounces daily between reported month-ends
#   - multiply by daily LBMA gold price to get gold AUM in USD
#   - take BTC market cap
#   - align them on the same calendar
#   - optionally resample to weekly/monthly
#   - winsorize BTC tail moves
#   - median-scale before fitting (so magnitudes are ~O(1))
# ============================================================================

def month_end_index(start_date: str, end_date: str) -> pd.DatetimeIndex:
    return pd.date_range(pd.to_datetime(start_date),
                         pd.to_datetime(end_date),
                         freq="ME")

def daily_index(start_date: str, end_date: str) -> pd.DatetimeIndex:
    return pd.date_range(pd.to_datetime(start_date),
                         pd.to_datetime(end_date),
                         freq="D")

def align_level_to_eom(series: pd.Series,
                       eom_idx: pd.DatetimeIndex) -> pd.Series:
    """
    For each month-end timestamp in eom_idx, grab the last known
    value in 'series' up to that date.
    """
    series = series.sort_index()
    out = []
    for ts in eom_idx:
        hist = series.loc[series.index <= ts]
        out.append(hist.iloc[-1] if len(hist) else np.nan)
    return pd.Series(out, index=eom_idx)

def _winsorize(s: pd.Series, p: float | None) -> pd.Series:
    """
    Clip extreme tails. Helps keep 'flash crash' type events
    from wrecking q(t) and regression stability.
    """
    if p is None or p <= 0:
        return s
    lo = s.quantile(p)
    hi = s.quantile(1 - p)
    return s.clip(lower=lo, upper=hi)

def load_gold_etf_holdings_monthly(path: str) -> pd.DataFrame:
    """
    Load gold ETF holdings in OUNCES and keep month-end levels.
    Result: DataFrame[gold_etf_ounces] indexed by month-end.
    """
    df = pd.read_excel(path)
    df = df[["Date", "Ounces"]].copy()

    df["Date"]   = pd.to_datetime(df["Date"], errors="coerce")
    df["Ounces"] = pd.to_numeric(df["Ounces"], errors="coerce")

    df = df.dropna().sort_values("Date")
    df = df[(df["Date"] >= pd.to_datetime(START_DATE)) &
            (df["Date"] <= pd.to_datetime(END_DATE))]

    eom_idx = month_end_index(START_DATE, END_DATE)
    s_oz = pd.Series(df["Ounces"].values, index=df["Date"].values)
    s_eom_oz = align_level_to_eom(s_oz, eom_idx)

    return pd.DataFrame({"gold_etf_ounces": s_eom_oz}, index=eom_idx)

def load_lbma_gold_price_daily(path: str) -> pd.DataFrame:
    """
    LBMA gold price per ounce in USD, daily.
    We forward-fill to get a full daily series.
    """
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    for rec in data:
        d = rec.get("d")
        vals = rec.get("v", [])
        usd = vals[0] if isinstance(vals, list) and len(vals) > 0 else None
        rows.append((d, usd))

    df = pd.DataFrame(rows, columns=["date", "usd_per_oz"])
    df["date"]        = pd.to_datetime(df["date"], errors="coerce")
    df["usd_per_oz"]  = pd.to_numeric(df["usd_per_oz"], errors="coerce")

    df = df.dropna().sort_values("date")
    df = df[(df["date"] >= pd.to_datetime(START_DATE)) &
            (df["date"] <= pd.to_datetime(END_DATE))]
    df = df.set_index("date")

    full_idx  = daily_index(START_DATE, END_DATE)
    df_daily  = df.reindex(full_idx).ffill()
    df_daily.columns = ["gold_usd_per_oz"]
    return df_daily

def load_bitcoin_daily(path: str) -> pd.DataFrame:
    """
    DAILY Bitcoin market cap (USD). We drop ETH completely.
    Column of interest: 'BTC / Market Cap (USD)' -> btc_mcap_usd.
    """
    encodings = ['utf-16','utf-16-le','utf-16-be','utf-8-sig','latin-1']
    df = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, sep="\t", encoding=enc)
            break
        except (UnicodeDecodeError, UnicodeError):
            continue
    if df is None:
        raise ValueError("Could not read BTC TSV.")

    need = ["Time", "BTC / Market Cap (USD)"]
    for c in need:
        if c not in df.columns:
            raise ValueError(f"Missing '{c}' in BTC TSV. Found {list(df.columns)}")

    df = df[need].copy()
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce")
    df["BTC / Market Cap (USD)"] = pd.to_numeric(
        df["BTC / Market Cap (USD)"], errors="coerce"
    )

    df = df.dropna(subset=["Time"]).sort_values("Time")
    df = df[(df["Time"] >= pd.to_datetime(START_DATE)) &
            (df["Time"] <= pd.to_datetime(END_DATE))]
    df = df.set_index("Time")

    # Resample to a regular daily calendar (last obs per day)
    daily = df.resample("D").last()

    # Winsorize BTC extremes to limit insane single-tick jumps
    if WINSOR_PCT and WINSOR_PCT > 0:
        daily["BTC / Market Cap (USD)"] = _winsorize(
            daily["BTC / Market Cap (USD)"], WINSOR_PCT
        )

    daily = daily.rename(columns={
        "BTC / Market Cap (USD)": "btc_mcap_usd"
    })

    return daily[["btc_mcap_usd"]]

def interpolate_ounces_daily(gold_ounces_monthly: pd.DataFrame) -> pd.DataFrame:
    """
    Gold ETF ounces are typically monthly (end-of-month).
    We linearly interpolate to daily to match Bitcoin frequency.
    """
    s_monthly = gold_ounces_monthly["gold_etf_ounces"].dropna()
    full_idx  = daily_index(START_DATE, END_DATE)
    s_daily   = s_monthly.reindex(full_idx)
    s_daily   = s_daily.interpolate(method="time")
    return pd.DataFrame({"gold_etf_ounces_daily": s_daily}, index=full_idx)

def build_gold_aum_daily(ounces_daily: pd.DataFrame,
                         gold_price_daily: pd.DataFrame) -> pd.DataFrame:
    """
    Gold AUM in USD = (ETF ounces) * (USD per ounce).
    That's our "capital stored in gold" series.
    """
    df = ounces_daily.join(gold_price_daily, how="inner")
    df["gold_etf_mcap_usd"] = (
        df["gold_etf_ounces_daily"] * df["gold_usd_per_oz"]
    )
    return df[["gold_etf_mcap_usd"]]

def build_panel_and_scale(gold_aum: pd.DataFrame,
                          btc: pd.DataFrame,
                          freq_name: str) -> Tuple[pd.DataFrame,
                                                   pd.DataFrame,
                                                   Dict]:
    """
    Create aligned panel for a given frequency (daily/weekly/monthly):
      - Join gold AUM and BTC mcap
      - Drop NaNs
      - MEDIAN-SCALE each series so typical value is ~1.0

    Scaling detail (our extension):
    We divide each asset by its own median market cap in that frequency
    window. This puts both assets on comparable O(1) scale so that
    parameter estimates (a,b,C) aren't dominated by the bigger series.
    We'll multiply back later to get USD predictions.
    """
    print(f"\n[{freq_name}] Building scaled panel...")

    df = gold_aum.join(btc[["btc_mcap_usd"]], how="inner").sort_index()

    panel_raw = df[["gold_etf_mcap_usd", "btc_mcap_usd"]].dropna(how="any").copy()

    eps = 1e-12
    med_gold = np.nanmedian(panel_raw["gold_etf_mcap_usd"])
    med_btc  = np.nanmedian(panel_raw["btc_mcap_usd"])

    # fallback in degenerate cases
    if not (np.isfinite(med_gold) and med_gold > 0): med_gold = 1.0
    if not (np.isfinite(med_btc)  and med_btc  > 0): med_btc  = 1.0

    panel_scaled = pd.DataFrame({
        "x": panel_raw["gold_etf_mcap_usd"] / (med_gold + eps),
        "y": panel_raw["btc_mcap_usd"]      / (med_btc  + eps),
    }, index=panel_raw.index)

    scale_info = {
        "gold_median_usd": med_gold,
        "btc_median_usd":  med_btc,
    }

    print(f"   {freq_name}: {len(panel_scaled)} observations")
    print(f"   Gold median:    {med_gold:.3g} USD")
    print(f"   Bitcoin median: {med_btc:.3g} USD")

    return panel_raw, panel_scaled, scale_info

def build_monthly():
    """
    Monthly panel:
    - Gold ETF AUM at month-end.
    - BTC market cap at month-end.
    - Build scaled panel.
    """
    gold_ounces_m = load_gold_etf_holdings_monthly(PATH_GOLD_ETF)
    lbma_price_d  = load_lbma_gold_price_daily(PATH_LBMA_JSON)
    lbma_monthly  = lbma_price_d.resample("ME").last()

    gold_aum = gold_ounces_m.join(lbma_monthly, how="inner")
    gold_aum["gold_etf_mcap_usd"] = (
        gold_aum["gold_etf_ounces"] * gold_aum["gold_usd_per_oz"]
    )
    gold_aum = gold_aum[["gold_etf_mcap_usd"]]

    btc_d       = load_bitcoin_daily(PATH_BTC_TSV)
    btc_monthly = btc_d.resample("ME").last()

    return build_panel_and_scale(gold_aum, btc_monthly, "MONTHLY")

def build_weekly():
    """
    Weekly panel:
    - First build daily gold AUM (interpolated ounces * daily price).
    - Then downsample both gold and BTC to W-FRI (Friday close).
    """
    gold_ounces_m = load_gold_etf_holdings_monthly(PATH_GOLD_ETF)
    gold_ounces_d = interpolate_ounces_daily(gold_ounces_m)
    lbma_price_d  = load_lbma_gold_price_daily(PATH_LBMA_JSON)
    gold_aum_d    = build_gold_aum_daily(gold_ounces_d, lbma_price_d)

    btc_d = load_bitcoin_daily(PATH_BTC_TSV)

    gold_aum_w = gold_aum_d.resample("W-FRI").last()
    btc_w      = btc_d.resample("W-FRI").last()

    return build_panel_and_scale(gold_aum_w, btc_w, "WEEKLY")

def build_daily():
    """
    Daily panel:
    - Daily gold AUM via interpolation+price.
    - Daily BTC mcap.
    """
    gold_ounces_m = load_gold_etf_holdings_monthly(PATH_GOLD_ETF)
    gold_ounces_d = interpolate_ounces_daily(gold_ounces_m)
    lbma_price_d  = load_lbma_gold_price_daily(PATH_LBMA_JSON)
    gold_aum_d    = build_gold_aum_daily(gold_ounces_d, lbma_price_d)

    btc_d = load_bitcoin_daily(PATH_BTC_TSV)

    return build_panel_and_scale(gold_aum_d, btc_d, "DAILY")

In [ ]:
# ============================================================================
# STEP 5: PARAMETER FITTING AND FORECAST
# ----------------------------------------------------------------------------
# THESIS REFERENCE: Section 3.2, Step 5
#
# We now estimate (a_i, b_i, C_ij) from the discrete VFGLVM equation:
#
#   x_i(k+1) ≈ a_i Z_i(k)
#              - b_i [Z_i(k)]^2
#              - Σ_{j≠i} C_ij Z_i(k) Z_j(k)
#
# For each asset i, we build a regression:
#   target  = x_i(k+1)             [what we want to predict]
#   columns = [ Z_i(k),            [linear growth term]
#               -(Z_i(k))^2,       [self-saturation term]
#               -(Z_i(k)*Z_j(k))   [cross-competition terms for j≠i]
#             ]
#
# Then solve via least squares: θ̂ = (B'B)^(-1) B'M
#
# This mirrors Eq. (15) and Eq. (17) in Gatabazi et al. (2019).
#
# ADAPTATION: Ridge Regularization (Our Extension)
# -------------------------------------------------
# Gatabazi et al. use vanilla least squares in Eq. (17):
#   θ̂ = (B'B)^(-1) B'M
#
# We add TINY ridge penalty (α=1e-5):
#   θ̂ = (B'B + αI)^(-1) B'M
#
# WHY RIDGE IS NEEDED:
# 1. COLLINEARITY IN FINANCIAL DATA:
#    - Z_gold and Z_btc can trend together (risk-on/risk-off regimes)
#    - Z_i and Z_i^2 are mechanically correlated
#    - Z_i*Z_j interaction terms compound correlations
#    → (B'B) can become near-singular, causing numerical instability
#
# 2. SCALE DIFFERENCES:
#    Even after median-scaling, gold (stable) and crypto (volatile)
#    have different variance properties that stress matrix inversion
#
# 3. NUMERICAL SAFETY:
#    α=1e-5 is chosen to be NEGLIGIBLE for parameter bias
#    (shrinks estimates by <0.001%) but prevents:
#    - Matrix inversion failures
#    - Extreme parameter values from near-collinearity
#    - Numerical overflow in forecasts
#
# This is standard practice when applying theoretical models to real data.
# The economic interpretation of parameters remains unchanged.
#
# ECONOMIC INTERPRETATION OF FITTED PARAMETERS:
# ----------------------------------------------
# After solving, we obtain for each asset i:
#
# a_i > 0:   Intrinsic growth (capital attraction when standalone)
#            Large a_i → Strong investor demand for asset i
#
# b_i > 0:   Self-saturation (crowding diminishes returns)
#            Large b_i → Asset quickly hits diminishing returns
#
# C_ij > 0:  Asset j crowds out asset i (substitution/competition)
#            Example: C_gold,btc > 0 means Bitcoin steals gold flows
#
# C_ij < 0:  Asset j helps asset i (complementarity/co-movement)
#            Example: C_gold,btc < 0 means Bitcoin and gold rise together
#
# C_ij ≈ 0:  Assets are independent (no capital reallocation between them)
#
# These parameters quantify the competitive dynamics between gold and crypto.
# ============================================================================

def fit_vfglvm_params(levels: List[np.ndarray],
                      Zq: List[np.ndarray],
                      ridge_alpha: float = 1e-5) -> VFGLVMParams:
    """
    Estimate VFGLVM parameters via ridge-regularized least squares.
    
    PROCESS:
    For each asset i:
      1. Build design matrix B with [Z_i, -Z_i^2, -Z_i*Z_j for j≠i]
      2. Set target vector M = x_i(k+1)
      3. Solve: θ̂ = (B'B + αI)^(-1) B'M
      4. Extract a_i, b_i, C_ij from θ̂
    
    PARAMETERS:
    - levels: List of raw scaled series [x_gold, x_btc]
    - Zq: List of background values from Step 4
    - ridge_alpha: Regularization strength (default 1e-5 for numerical stability)
    
    RETURNS:
    - VFGLVMParams object with fitted a, b, C arrays
    """
    n = len(levels)
    T = min(len(x) for x in levels)

    X0 = np.vstack([_as_array(x) for x in levels])[:, :T]  # raw scaled x_i(k)
    Z  = np.vstack([_as_array(z) for z in Zq])[:, :T]      # background Z_i(k)

    # Use indices k where we have both Z_i(k) and x_i(k+1)
    idx_all = np.arange(1, T-1)
    mask = np.ones(len(idx_all), dtype=bool)
    for i in range(n):
        mask &= np.isfinite(Z[i, idx_all]) & np.isfinite(X0[i, idx_all+1])
    idx = idx_all[mask]
    
    if len(idx) < 5:
        raise RuntimeError(f"Not enough usable points to fit VFGLVM ({len(idx)})")

    a_hat = np.zeros(n)
    b_hat = np.zeros(n)
    C_hat = np.zeros((n, n))

    # Fit each asset's equation separately
    for i in range(n):
        zi = Z[i, idx]

        # Build design matrix B:
        # Column 1: Z_i(k)           → coefficient a_i
        # Column 2: -Z_i(k)^2        → coefficient b_i  
        # Column 3+: -Z_i(k)*Z_j(k)  → coefficients C_ij for j≠i
        cols = [zi, -(zi**2)]

        # Add cross terms with all other assets
        for j in range(n):
            if j == i:
                continue
            zj = Z[j, idx]
            cols.append(-(zi * zj))

        Bi = np.column_stack(cols)
        Mi = X0[i, idx+1]  # x_i(k+1) is the target

        # Ridge-regularized least squares
        theta = _ridge(Bi, Mi, alpha=ridge_alpha)

        # Extract parameters
        a_hat[i] = theta[0]
        b_hat[i] = theta[1]

        c_idx = 2
        for j in range(n):
            if j == i:
                continue
            C_hat[i, j] = theta[c_idx]
            c_idx += 1

    return VFGLVMParams(a=a_hat, b=b_hat, C=C_hat)

def vfglvm_fitted_values(params: VFGLVMParams,
                         levels: List[np.ndarray],
                         Zq: List[np.ndarray]) -> List[np.ndarray]:
    """
    Produce in-sample 1-step-ahead fitted values:
    ̂x_i(k+1) = a_i Z_i(k) - b_i [Z_i(k)]^2 - Σ_j C_ij Z_i(k)Z_j(k)
    
    This generates the model's predictions that we compare to actual values
    for computing MAPE, RMSE, etc.
    
    PARAMETERS:
    - params: Fitted VFGLVMParams from fit_vfglvm_params
    - levels: List of raw scaled series (for dimensionality)
    - Zq: List of background values (the state variables)
    
    RETURNS:
    - List of fitted series in scaled units (will be rescaled to USD later)
    """
    n = len(levels)
    T = len(levels[0])
    fitted = [np.full(T, np.nan) for _ in range(n)]

    for k in range(1, T):
        z_k = np.array([Zq[i][k] for i in range(n)])
        if not np.all(np.isfinite(z_k)):
            continue
        for i in range(n):
            # Apply VFGLVM equation with fitted parameters
            pred = params.a[i] * z_k[i] - params.b[i] * (z_k[i]**2)
            for j in range(n):
                if j != i:
                    pred -= params.C[i, j] * z_k[i] * z_k[j]
            if k+1 < T:
                fitted[i][k+1] = pred
    return fitted

def _default_max_lag(n_obs: int) -> int:
    """
    Cap how far back we look in the fractional accumulation.
    
    RATIONALE:
    - Memory is long but not infinite
    - Computational efficiency (O(n²) otherwise)
    - Numerical stability (Gamma weights decay to zero anyway)
    
    HEURISTIC:
    - Short series (≤200 obs): 36-step memory (~1 month daily data)
    - Long series (>200 obs): 156-step memory (~6 months daily data)
    """
    return 36 if n_obs <= 200 else 156

def metrics_levels(y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    """
    Compute error metrics for model evaluation.
    
    METRICS:
    - MAPE (Mean Absolute Percentage Error): Primary metric used by
      Gatabazi et al. (2019) to judge model quality
      * MAPE < 10%: Highly accurate
      * MAPE 10-20%: Good accuracy
      * MAPE 20-50%: Reasonable accuracy
      * MAPE > 50%: Poor accuracy
    
    - RMSE (Root Mean Squared Error): Economic interpretability in $B
      Shows average forecast error in dollar terms
    
    - MAE (Mean Absolute Error): Robust to outliers, complements RMSE
    
    These combine statistical precision (MAPE) with economic
    interpretability (RMSE in billions USD).
    """
    mask = np.isfinite(y_true) & np.isfinite(y_pred)
    if not mask.any():
        return dict(RMSE=np.nan, MAE=np.nan, MAPE=np.nan)

    yt = y_true[mask]
    yp = y_pred[mask]
    err = yt - yp

    rmse = float(np.sqrt(np.mean(err**2)))
    mae  = float(np.mean(np.abs(err)))

    with np.errstate(divide='ignore', invalid='ignore'):
        ape = np.abs(err) / np.abs(yt)
        ape = ape[np.isfinite(ape)]
        mape = float(np.mean(ape)*100) if len(ape) > 0 else np.nan

    return dict(RMSE=rmse, MAE=mae, MAPE=mape)

def approx_lyapunov(params: VFGLVMParams) -> np.ndarray:
    """
    OPTIONAL: Simplified Lyapunov-style indicator for chaos detection.
    
    Gatabazi et al. (2019) discuss Lyapunov exponents of the fitted LV
    system to argue the dynamics are chaotic. In their reported results,
    the exponents essentially match the leading growth terms a_i.
    
    INTERPRETATION:
    - All λ_i > 0: Chaotic dynamics (sensitive dependence on initial conditions)
    - Mixed signs: Complex dynamics with some stability
    - All λ_i < 0: Stable equilibrium (no chaos)
    
    For gold vs crypto:
    - Positive Lyapunov exponents confirm market dynamics are chaotic
    - This justifies the need for adaptive memory (variable q(t))
    - Standard fixed-parameter models fail in chaotic systems
    
    NOTE: This is NOT a full Lyapunov calculation (which requires
    trajectory analysis). It's a quick heuristic based on linearization
    at the trivial equilibrium, matching Gatabazi et al.'s approach.
    """
    return params.a.copy()

In [ ]:
# ============================================================================
# FIT AND EVALUATE VFGLVM FOR ONE FREQUENCY
# ============================================================================

def fit_vfglvm_frequency(panel_scaled: pd.DataFrame,
                         panel_raw: pd.DataFrame,
                         scale_info: Dict,
                         freq_name: str,
                         window_q: int,
                         smooth_ma_q: int) -> Dict:
    """
    Complete VFGLVM pipeline for one frequency.
    
    PROCESS:
    1. Take scaled series for gold (x) and bitcoin (y)
    2. Estimate q(t) from rolling slopes (Step 3)
    3. Build fractional accumulation X^{[q(t)]} and background Z (Steps 2 & 4)
    4. Fit VFGLVM params (Step 5)
    5. Generate 1-step-ahead fitted series
    6. Unscale back to USD using stored medians
    7. Compute error metrics (MAPE, RMSE)
    
    PARAMETERS:
    - panel_scaled: Median-normalized asset levels
    - panel_raw: Original USD levels (for final comparison)
    - scale_info: Median values used for normalization
    - freq_name: "DAILY", "WEEKLY", or "MONTHLY" (for logging)
    - window_q: Rolling window for q(t) estimation
    - smooth_ma_q: Smoothing window for q(t)
    
    RETURNS:
    - Dictionary with fitted values, parameters, metrics, and diagnostics
    """

    print(f"\n[{freq_name}] Fitting VFGLVM...")

    # Levels in scaled space: gold = x, bitcoin = y
    levels = [
        _as_array(panel_scaled["x"].values),
        _as_array(panel_scaled["y"].values),
    ]
    X0_full = np.vstack(levels)

    # --- Step 3: estimate q(t) from market dynamics
    q_t = estimate_q_t(X0_full, window=window_q, smooth_ma=smooth_ma_q)
    if not np.isfinite(q_t).any():
        # fallback to constant memory
        q_t = np.full(len(panel_scaled), 0.5)

    # --- Step 2 + Step 4: fractional accumulation + background value
    max_lag = _default_max_lag(len(panel_scaled))
    Xq, Zq = build_Z_q(levels, q_t, max_lag=max_lag)

    # --- Step 5: fit LV-style discrete dynamics
    params = fit_vfglvm_params(levels, Zq, ridge_alpha=1e-5)

    # --- Create in-sample fitted paths in *scaled units*
    fitted_list = vfglvm_fitted_values(params, levels, Zq)

    # --- Unscale back to USD using medians from this frequency
    med_gold = scale_info["gold_median_usd"]
    med_btc  = scale_info["btc_median_usd"]

    gold_fitted_raw = fitted_list[0] * med_gold
    btc_fitted_raw  = fitted_list[1] * med_btc

    df_out = pd.DataFrame({
        "gold_actual": panel_raw["gold_etf_mcap_usd"],
        "gold_fitted": gold_fitted_raw,
        "btc_actual":  panel_raw["btc_mcap_usd"],
        "btc_fitted":  btc_fitted_raw,
    }, index=panel_raw.index)

    # --- Error metrics (MAPE like Gatabazi et al. (2019), plus RMSE in $B)
    gold_metrics = metrics_levels(panel_raw["gold_etf_mcap_usd"].values,
                                  gold_fitted_raw)
    btc_metrics  = metrics_levels(panel_raw["btc_mcap_usd"].values,
                                  btc_fitted_raw)

    print(f"   Gold:    MAPE={gold_metrics['MAPE']:.2f}%, "
          f"RMSE=${gold_metrics['RMSE']/1e9:.2f}B")
    print(f"   Bitcoin: MAPE={btc_metrics['MAPE']:.2f}%, "
          f"RMSE=${btc_metrics['RMSE']/1e9:.2f}B")

    return {
        "df_fit": df_out,
        "params": params,
        "q_t": q_t,
        "gold_mape": gold_metrics['MAPE'],
        "gold_rmse": gold_metrics['RMSE'],
        "btc_mape": btc_metrics['MAPE'],
        "btc_rmse": btc_metrics['RMSE'],
        "lyapunov_like": approx_lyapunov(params),  # optional chaos hint
    }

In [ ]:
# ============================================================================
# PLOTTING: 6-PANEL (Gold row, Bitcoin row) ACROSS FREQUENCIES
# ============================================================================

def plot_6_panel(results_daily,
                 results_weekly,
                 results_monthly):
    """
    Make the 2x3 grid:
        Top row  = Gold ETF AUM (Daily, Weekly, Monthly)
        Bottom   = Bitcoin Market Cap (Daily, Weekly, Monthly)

    Each subplot shows:
        - Actual (in USD billions)
        - VFGLVM fitted (1-step-ahead in USD billions)
        - Text box with MAPE and RMSE
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))

    # Row 1: Gold
    for i, (freq_name, results) in enumerate([
        ("Daily",   results_daily),
        ("Weekly",  results_weekly),
        ("Monthly", results_monthly),
    ]):
        ax = axes[0, i]
        df = results["df_fit"]

        ax.plot(df.index,
                df["gold_actual"]/1e9,
                label="Actual",
                lw=2,
                alpha=0.7,
                color='goldenrod')
        ax.plot(df.index,
                df["gold_fitted"]/1e9,
                label="VFGLVM",
                lw=1.5,
                alpha=0.9,
                linestyle='--',
                color='darkgoldenrod')

        ax.set_title(f"Gold ETF AUM - {freq_name}",
                     fontsize=11,
                     fontweight='bold')
        ax.set_ylabel("Billions USD", fontsize=10)
        ax.legend(loc='best', fontsize=9)
        ax.grid(alpha=0.3)

        ax.text(
            0.02, 0.98,
            f"MAPE: {results['gold_mape']:.1f}%\n"
            f"RMSE: ${results['gold_rmse']/1e9:.1f}B",
            transform=ax.transAxes,
            fontsize=9,
            verticalalignment='top',
            bbox=dict(boxstyle='round',
                      facecolor='wheat',
                      alpha=0.6)
        )

    # Row 2: BTC
    for i, (freq_name, results) in enumerate([
        ("Daily",   results_daily),
        ("Weekly",  results_weekly),
        ("Monthly", results_monthly),
    ]):
        ax = axes[1, i]
        df = results["df_fit"]

        ax.plot(df.index,
                df["btc_actual"]/1e9,
                label="Actual",
                lw=2,
                alpha=0.7,
                color='steelblue')
        ax.plot(df.index,
                df["btc_fitted"]/1e9,
                label="VFGLVM",
                lw=1.5,
                alpha=0.9,
                linestyle='--',
                color='navy')

        ax.set_title(f"Bitcoin Market Cap - {freq_name}",
                     fontsize=11,
                     fontweight='bold')
        ax.set_ylabel("Billions USD", fontsize=10)
        ax.set_xlabel("Date", fontsize=10)
        ax.legend(loc='best', fontsize=9)
        ax.grid(alpha=0.3)

        ax.text(
            0.02, 0.98,
            f"MAPE: {results['btc_mape']:.1f}%\n"
            f"RMSE: ${results['btc_rmse']/1e9:.1f}B",
            transform=ax.transAxes,
            fontsize=9,
            verticalalignment='top',
            bbox=dict(boxstyle='round',
                      facecolor='lightblue',
                      alpha=0.6)
        )

    plt.tight_layout()
    plt.savefig(OUTDIR / "vfglvm_6panel_comparison_gold_vs_btc.png",
                dpi=300,
                bbox_inches="tight")
    print(f"\n✅ Saved: {OUTDIR / 'vfglvm_6panel_comparison_gold_vs_btc.png'}")
    plt.show()

In [ ]:
# ============================================================================
# MAIN PIPELINE
# ============================================================================

def main():
    print("\n" + "="*70)
    print(" VFGLVM MULTI-FREQUENCY COMPARISON")
    print(" GOLD ETF AUM vs BITCOIN ONLY")
    print("="*70)

    # --- Build panels for each freq
    panel_raw_monthly, panel_scaled_monthly, scale_monthly = build_monthly()
    panel_raw_weekly,  panel_scaled_weekly,  scale_weekly  = build_weekly()
    panel_raw_daily,   panel_scaled_daily,   scale_daily   = build_daily()

    # --- Fit VFGLVM for each freq.
    # We pick slightly different q(t) window/smoothing per freq.
    results_monthly = fit_vfglvm_frequency(
        panel_scaled_monthly, panel_raw_monthly, scale_monthly,
        "MONTHLY", window_q=12, smooth_ma_q=4
    )
    results_weekly = fit_vfglvm_frequency(
        panel_scaled_weekly, panel_raw_weekly, scale_weekly,
        "WEEKLY", window_q=20, smooth_ma_q=6
    )
    results_daily = fit_vfglvm_frequency(
        panel_scaled_daily, panel_raw_daily, scale_daily,
        "DAILY", window_q=30, smooth_ma_q=8
    )

    # --- Visual comparison grid (6 subplots)
    plot_6_panel(results_daily, results_weekly, results_monthly)

    # --- PERFORMANCE SUMMARY TABLE
    print("\n" + "="*70)
    print(" PERFORMANCE SUMMARY")
    print("="*70)

    summary = pd.DataFrame([
        {"Frequency": "Monthly", "Asset": "Gold",
         "MAPE_%": results_monthly["gold_mape"],
         "RMSE_B": results_monthly["gold_rmse"]/1e9},

        {"Frequency": "Monthly", "Asset": "Bitcoin",
         "MAPE_%": results_monthly["btc_mape"],
         "RMSE_B": results_monthly["btc_rmse"]/1e9},

        {"Frequency": "Weekly",  "Asset": "Gold",
         "MAPE_%": results_weekly["gold_mape"],
         "RMSE_B": results_weekly["gold_rmse"]/1e9},

        {"Frequency": "Weekly",  "Asset": "Bitcoin",
         "MAPE_%": results_weekly["btc_mape"],
         "RMSE_B": results_weekly["btc_rmse"]/1e9},

        {"Frequency": "Daily",   "Asset": "Gold",
         "MAPE_%": results_daily["gold_mape"],
         "RMSE_B": results_daily["gold_rmse"]/1e9},

        {"Frequency": "Daily",   "Asset": "Bitcoin",
         "MAPE_%": results_daily["btc_mape"],
         "RMSE_B": results_daily["btc_rmse"]/1e9},
    ])

    print("\n", summary.round(2).to_string(index=False))
    summary.to_csv(OUTDIR / "performance_summary_gold_vs_btc.csv",
                   index=False)

    # --- Save fitted paths (actual vs fitted)
    results_monthly["df_fit"].to_csv(
        OUTDIR / "fitted_monthly_gold_vs_btc.csv")
    results_weekly["df_fit"].to_csv(
        OUTDIR / "fitted_weekly_gold_vs_btc.csv")
    results_daily["df_fit"].to_csv(
        OUTDIR / "fitted_daily_gold_vs_btc.csv")

    # --- PARAMETER COMPARISON TABLE
    # Interpretability:
    #   a_gold ~ intrinsic growth of gold AUM
    #   a_bitcoin ~ intrinsic growth of BTC cap
    #   b_* ~ self-crowding / saturation
    #   C_bitcoin_to_gold ~ effect of BTC on gold
    #   C_gold_to_bitcoin ~ effect of gold on BTC
    print("\n" + "="*70)
    print(" PARAMETER COMPARISON (Gold = series x, Bitcoin = series y)")
    print("="*70)

    param_df = pd.DataFrame({
        "Frequency": ["Monthly", "Weekly", "Daily"],
        "a_gold":    [results_monthly["params"].a[0],
                      results_weekly["params"].a[0],
                      results_daily["params"].a[0]],
        "a_bitcoin": [results_monthly["params"].a[1],
                      results_weekly["params"].a[1],
                      results_daily["params"].a[1]],

        "b_gold":    [results_monthly["params"].b[0],
                      results_weekly["params"].b[0],
                      results_daily["params"].b[0]],
        "b_bitcoin": [results_monthly["params"].b[1],
                      results_weekly["params"].b[1],
                      results_daily["params"].b[1]],

        # C[0,1] = impact of Bitcoin on Gold (crowd-out if +)
        # C[1,0] = impact of Gold on Bitcoin
        "C_bitcoin_to_gold": [results_monthly["params"].C[0,1],
                              results_weekly["params"].C[0,1],
                              results_daily["params"].C[0,1]],
        "C_gold_to_bitcoin": [results_monthly["params"].C[1,0],
                              results_weekly["params"].C[1,0],
                              results_daily["params"].C[1,0]],

        # optional chaos-ish "growth rates"
        "approx_lyapunov_gold": [results_monthly["lyapunov_like"][0],
                                 results_weekly["lyapunov_like"][0],
                                 results_daily["lyapunov_like"][0]],
        "approx_lyapunov_btc":  [results_monthly["lyapunov_like"][1],
                                 results_weekly["lyapunov_like"][1],
                                 results_daily["lyapunov_like"][1]],
    })

    print("\n", param_df.round(6).to_string(index=False))
    param_df.to_csv(OUTDIR / "parameter_comparison_gold_vs_btc.csv",
                    index=False)

    print("\n✅ All outputs saved to:", OUTDIR)
    print("="*70 + "\n")

if __name__ == "__main__":
    main()